## K Nearest Neighbor on UMAP Data + HOG +SUMY SUMX

In [21]:
# Load packages
import pandas as pd
import numpy as np
import cv2 as cv
import time

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [3]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
#previously mapped training data by UMAP
umap_train = pd.read_csv('umap_train_gray_small_hog_xy_3d.csv')

**previously UMAP architecture**

**transform test data**

In [6]:
test_data = pd.read_csv('test_gray_small.csv')

In [7]:
sumy_df = pd.read_csv('histy_test.csv')

In [8]:
sumx_df = pd.read_csv('histx_test.csv')

In [9]:
hog_df = pd.read_csv('hog_fd_test.csv')

In [10]:
test_data.shape, sumy_df.shape, sumx_df.shape, hog_df.shape

((965, 1201), (965, 41), (965, 31), (965, 1729))

In [11]:
final_df = pd.concat([test_data, sumy_df.iloc[:,1:],sumx_df.iloc[:,1:],  hog_df.iloc[:,1:] ], axis=1)

In [12]:
final_df.shape

(965, 2999)

## LinearSVC fit in 2D

In [13]:
#previously mapped training data by UMAP
umap_train = pd.read_csv('umap_train_gray_small_hog_xy_2d.csv')

In [14]:
umap_train.head()

,label,embedding x,embedding y
0,0.0,-10.455663,2.890634
1,2.0,2.295300,-2.460490
2,2.0,-2.464793,-5.478656
3,3.0,6.898623,-3.838779
4,1.0,-5.673230,-5.388057


In [15]:
import joblib
# Load the saved model
umap_model = joblib.load('umap_model_gray_small_hog_xy_2d.pkl')

# Transform new data
#slicing is to remove label in column 0
embedding = umap_model.transform(final_df.iloc[:,1:])

In [16]:
#previously mapped test data by UMAP
umap_test = embedding

In [17]:
#train_df.iloc[:, 1:].values.astype(np.float32)
x_train_u = umap_train.iloc[:, 1:].values.astype(np.float32)
y_train_u =umap_train['label'].values

In [18]:
x_test_u = umap_test
y_test_u =final_df['label'].values

In [29]:
# Classification with a linear SVM
percent =100
start_time = time.time()
svc = LinearSVC(dual=False, random_state=123)
params_grid = {"C": [10 ** k for k in range(-3, 4)]}
clf = GridSearchCV(svc, params_grid)
clf.fit(x_train_u, y_train_u)
print('percent complete: ',percent, end =" - %s seconds -\n" % \
              (time.time() - start_time), flush=True)


percent complete:  100 - 0.19713950157165527 seconds -


In [30]:
import pickle

# Save the trained model as a pickle string.
saved_model = pickle.dumps(clf)

# Save the model to disk
with open('lsvc_umap_gray_small_hog_xy_2d.pkl', 'wb') as file:
    pickle.dump(clf, file)
    
    # Load the model from disk
with open('lsvc_umap_gray_small_hog_xy_2d.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [31]:
y_pred_u = loaded_model.predict(x_test_u)

In [32]:
from sklearn.metrics import accuracy_score

# Assuming y_test_u are the true labels for the test set
accuracy = accuracy_score(y_test_u, y_pred_u)

print(f'The accuracy of the LinearSVC model is {accuracy:.2f}')


The accuracy of the LinearSVC model is 0.28


**No Good**